# NOTE

Please make certain that the `DBNAME` variable in `lib/init.py` is set to

    DBNAME = 'CDR'
    
to use this notebook

# Snorkel Workshop: Extracting `Chemical-Disease` Relations (CDR) from PubMed Abstracts
## Part 2: Writing  Labeling Functions

In Snorkel, our primary interface through which we provide training signal to the end extraction model we are training is by writing **labeling functions (LFs)** (as opposed to hand-labeling massive training sets).  We'll go through some examples for our `Chemical-Disease` extraction task below.

A labeling function isn't anything special. It's just a Python function that accepts a `Candidate` as the input argument and returns `1` if it says the `Candidate` should be marked as true, `-1` if it says the `Candidate` should be marked as false, and `0` if it doesn't know how to vote and abstains. In practice, many labeling functions are unipolar: it labels only `1`s and `0`s, or it labels only `-1`s and `0`s.

Recall that our goal is to ultimately train a high-performance classification model that predicts which of our `Candidate`s are true mentions of chemical-disease relations.  It turns out that we can do this by writing potentially low-quality labeling functions and aggregating them using Snorkel into higher quality labels!

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import re
import sys
import numpy as np

# Connect to the database backend and initalize a Snorkel session
from lib.init import *
from lib.scoring import *
from lib.lf_factories import *

from snorkel.lf_helpers import test_LF
from snorkel.annotations import load_gold_labels
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

# initialize our candidate type definition
ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

# I. Background

## A. Task Definition


The CDR task is comprised of three sets of 500 documents each, called training, development, and test. A document consists of the title and abstract of an article from [PubMed](https://www.ncbi.nlm.nih.gov/pubmed/), an archive of biomedical and life sciences journal literature. The documents have been hand-annotated with

* Mentions of chemicals and diseases along with their [MESH](https://meshb.nlm.nih.gov/#/fieldSearch) IDs, canonical IDs for medical entities. For example, mentions of "warfarin" in two different documents will have the same ID.
* Chemical-disease relations at the document-level. That is, if some piece of text in the document implies that a chemical with MESH ID `X` induces a disease with MESH ID `Y`, the document will be annotated with `Relation(X, Y)`

We'll be writing an application to extract mentions of `chemical-induced-disease` relationships from Pubmed abstracts, as per the BioCreative CDR Challenge. 


## B. Preprocessing the Database

In a real application, there is a lot of data preparation, parsing, and database loading that needs to be completed before we dive into writing labeling functions. Here we've pre-generated a database instance for you. All _candidates_ and _gold labels_ (i.e., human-generated labels) are queried from this database for use in the the tutorial. 

Download and launch our preprocessing tutorial <a href="https://github.com/HazyResearch/snorkel/tree/master/tutorials/workshop">Workshop 5 Advanced Preprocessing</a> from our GitHub page for more details on how this database is built.

## C. Using a _Development Set_ of Human-labeled Data

In our setting, we will use the phrase _development set_ to refer to a set of examples (here, a subset of our training set) which we label by hand and use to help us develop and refine labeling functions.  Unlike the _test set_, which we do not look at and use for final evaluation, we can inspect the development set while writing labeling functions. This is a list of `{-1,1}` labels.

In [ ]:
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)


## D. Data Exploration

### 1. Viewing Sentences
How do we come up with good keywords and patterns to encode as labeling functions? One way is to manually explore our training data. Here we load a subset of our training candidates into a `SentenceNgramViewer` object to examine candidates in their parent context. Our goal is to build an intuition for patterns and keywords that are predictive of a candidate's true label. 

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# load our list of training & development candidates
train_cands = session.query(Candidate).filter(Candidate.split == 0).order_by(Candidate.id).all()
dev_cands   = session.query(Candidate).filter(Candidate.split == 1).order_by(Candidate.id).all()

SentenceNgramViewer(train_cands[0:500], session, n_per_page=1)


### 2. Exploring Gold Labeled Data

We can use another visualization tool to explore a small set of labeled candidates to help build out inutution for posiive and negative class examples.  

In [ ]:
from lib.viz import *

POSITIVE, UNLABLED, NEGATIVE = 1, 0, -1
view_labeled_candidates(dev_cands, L_gold_dev, label=POSITIVE, n_max=5)
    

## E. Labeling Function Metrics

### 1. Coverage
One simple metric we can compute quickly is our _coverage_, the number of candidates labeled by our LF $\lambda_j$, on our training set (or any other set).

\begin{eqnarray}
coverage_{\lambda_j} = \frac{1}{N}\sum_{i=1}^{N} \unicode{x1D7D9}({\lambda_j}{(x_i) \in \{-1,1\}})
\label{eq3}
\end{eqnarray}


### 2. Precision / Recall / F1
If we have gold labeled data, we can also compute standard precision, recall, and F1 metrics for the output of a single labeling function. These metrics are computed over 4 _error buckets_: _True Positives_ (tp), _False Positives_ (fp), _True Negatives_ (tn), and _False Negatives_ (fn).

\begin{equation*}
precision = \frac{tp}{(tp + fp)}
\end{equation*}

\begin{equation*}
recall = \frac{tp}{(tp + fn)}
\end{equation*}

\begin{equation*}
F1 = 2 \cdot \frac{ (precision \cdot recall)}{(precision + recall)}
\end{equation*}

# II. Labeling Functions

## A. Pattern Matching Labeling Functions

One powerful form of labeling function design is defining sets of keywords or regular expressions that, as a human labeler, you know are correlated with the true label. In the terminology of [Bayesian inference](https://en.wikipedia.org/wiki/Statistical_inference#Bayesian_inference), this can be thought of as defining a [_prior_](https://en.wikipedia.org/wiki/Prior_probability) over your word features. 

For example, we could define a dictionary of terms that occur between chemical and disease names in a candidate. One simple dictionary of terms indicating a true relation could be:
    
    treatment = {'treatment'}
 
We can then write a labeling function that checks for a match with these terms in the text that occurs between person names.

    def LF_treatment_terms_between(c):
        return 1 if len(treat.intersection(get_between_tokens(c))) > 0 else 0
        
The idea is that we can easily create dictionaries that encode themes or categories descibing different types of interactions between a `chemical` and `disease` and then use these objects to _weakly supervise_ our classification task.

**IMPORTANT** Good labeling functions manage a trade-off between high coverage and high precision. When constructing your dictionaries, think about building larger, noiser sets of terms instead of relying on 1 or 2 keywords. Sometimes a single word can be very predictive but it's almost always better to define something more general, such as a regular expressions or dictionaries of many related words. 
    

### 1. Labeling Function Factories
The above is a reasonable way to write labeling functions. However, this type of design pattern is so common that we rely on another abstraction to help us build LFs more quickly: _labeling function factories_. Factories accept simple inputs, like dictionaries or a set of regular expressions, and automatically builds labeling functions for you.

The `MatchTerms` and `MatchRegex` factories require a few parameter definitions to setup:
    
    name:    a string that describes the category of terms/regular expressons
    label:   patterns correlate with a True or False label (1 or -1) 
    search:  search a specific part of the sentence ('left'|'right'|'between'|'sentence')
    window:  the length of tokens to match against for ('left'|'right') search spaces 

### 2. Term Matching Factory
We illustrate below how you can use the `MatchTerms` factory to create and test an LF on training candidates. When examining candidates in the `SentenceNgramViewer`, notice that treatment always occurs between disease and chemical names. That is the supervision signal encoded by this LF!

In [ ]:
terms = {'treatment'}

# we'll initialize our LFG and test its coverage on training candidates
LF_treatment_btw = MatchTerms(name='treatment', terms=terms, label=1, search='between').lf()

# what candidates are covered by this LF?
labeled = coverage(session, LF_treatment_btw, split=0)

# now let's view what this LF labeled
SentenceNgramViewer(labeled, session, n_per_page=1)


#### Viewing Error Buckets
If we have gold labeled data, we can evaluate formal metrics. It's useful to view specific errors for a given LF input in the `SentenceNgramViewer`.

Below, we'll compute our empirical scores using human-labeled development set data and then look at any false positive matches by our `LF_treatment_btw` LF. We can see below from our scores that this LF isn't very accurate -- only 24% precision!

In [ ]:
tp, fp, tn, fn = error_analysis(session, LF_treatment_btw, split=1, gold=L_gold_dev)

# now let's view what this LF labeled
SentenceNgramViewer(tp, session, n_per_page=1)


### 3. Regular Expression Factory

Sometimes we want to express more generic textual patterns to match against candidates. Perhaps we want to match a specific phrase like '`did not`' or look for modifier suffixes like `[-]*induced` .

We can generate this supervision in the same way as above using sets of [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) -- a formal language for string matching.

In [ ]:
negation_rgxs = {'did not'}

LF_negation_btw = MatchRegex(name='negation', rgxs=negation_rgxs, label=-1, search='between').lf()
labeled = coverage(session, LF_negation_btw, split=1)

# now let's view what this LF labeled
SentenceNgramViewer(labeled, session, n_per_page=1)

In [ ]:
tp, fp, tn, fn = error_analysis(session, LF_negation_btw, split=1, gold=L_gold_dev)
SentenceNgramViewer(tn, session, n_per_page=1)

## B. Distant Supervision Labeling Functions

We'll use the [Comparative Toxicogenomics Database (CTD)](http://ctdbase.org/) for distant supervision. The CTD lists chemical-condition entity pairs under three categories: 
- **Therapy**: the chemical treats the condition
- **Marker**: the chemical is typically present with the condition
- **Unspecified**: ...unspecified

We provide 3 helper functions to test if a candidate is in one of the categories. These use `c.get_cids()` to access each entity mention's normalized concept identifier ([MeSH codes](https://en.wikipedia.org/wiki/List_of_MeSH_codes))  

In [ ]:
import bz2
from six.moves.cPickle import load

with open('data/ctd.pkl', 'rb') as ctd_f:
    ctd_unspecified, ctd_therapy, ctd_marker = load(ctd_f)
    
def cand_in_ctd_unspecified(c):
    return 1 if c.get_cids() in ctd_unspecified else 0

def cand_in_ctd_therapy(c):
    return 1 if c.get_cids() in ctd_therapy else 0

def cand_in_ctd_marker(c):
    return 1 if c.get_cids() in ctd_marker else 0


In [ ]:
def LF_ctd_marker(c):
    return 1 if cand_in_ctd_marker(c) else 0

score(session, LF_ctd_marker, split=1, gold=L_gold_dev)

SentenceNgramViewer(labeled, session, n_per_page=1)

## C. Writing Custom Labeling Functions

The strength of LFs is that you can write any arbitrary function and use it to supervise a classification task. This approach can combine many of the same strategies discussed above or encode other information. 

For example, we observe that when mentions of chemical and disease names occur far apart in a sentence, this is a good indicator that the candidate's label is False.

In [ ]:
def LF_too_far_apart(c):
    """Entity mentions occur at a distance > 10 words"""
    return -1 if len(list(get_between_tokens(c))) > 10 else 0


In [ ]:
labeled = coverage(session, LF_too_far_apart, split=1)

score(session, LF_too_far_apart, split=1, gold=L_gold_dev)

## D. Composing Labeling Functions

Another useful technique for writing LFs is composing multiple, weaker LFs together. For example, our `LF_treatment_btw` example above has low precision.  Instead of modifying `LF_treatment_btw`, we'll compose it with our `LF_too_far_apart` from above.

    LF_treatment_btw                              Precision: 27.7 | TP: 14 | FP: 45  
    LF_treatment_btw AND NOT LF_too_far_apart     Precision: 34.6 | TP: 9  | FP: 17 

We dropped 5 true candidates, but we cut our false positive rate by 28 candidates !

In [ ]:
def LF_treatment_AND_too_far_apart(c):
    return 1 if LF_too_far_apart(c) != -1 and LF_treatment_btw(c) == 1 else 0

score(session, LF_treatment_AND_too_far_apart, split=1, gold=L_gold_dev)

# III. Development Sandbox
----

## A. Writing Your Own Labeling Functions

Using the information above, write your own labeling functions for this task. 

In [ ]:
#
# PLACE YOUR LFs HERE
#

## B. Applying Labeling Functions
---

Next, we need to actually run the LFs over **all** of our training candidates, producing a set of `Labels` and `LabelKeys` (just the names of the LFs) in the database.  We'll do this using the `LabelAnnotator` class, a UDF which we will again run with `UDFRunner`.

### 1. Preparing your Labeling Functions

First we put all our labeling functions into list:

In [ ]:
LFs = [
    # Place your lf function variable names here
]


Then we setup the label annotator class:

In [ ]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)

### 2. Generating the Label Matrix

In [ ]:
np.random.seed(1701)

%time L_train = labeler.apply(split=0, parallelism=1)
print(L_train.shape)

%time L_dev = labeler.apply_existing(split=1, parallelism=1)
print(L_dev.shape)

In [ ]:
L_train.lf_stats(session)

### 3. Label Matrix Empirical Accuracies

If we have a small set of human-labeled data, we can also check the empirical accuracies of LFs.

In [ ]:
L_dev.lf_stats(session, labels=L_gold_dev.toarray().ravel())

## C. Iterating on Labeling Function Design

When writing labeling functions, you will want to iterate on the process outlined above several times. You should focus on tuning individual LFs, based on emprical accuracy metrics, and adding new LFs to improve coverage. 